In [10]:
# encoding: utf-8

# https://stackoverflow.com/questions/26567843/reading-the-mail-content-of-an-mbox-file-using-python-mailbox
# https://www.debuggex.com/cheatsheet/regex/python

import unicodecsv as csv
import re
import sys

stdout = sys.stdout
reload(sys)
sys.stdout = stdout

# sys.setdefaultencoding("ISO-8859-1")

In [11]:
def clean_content(content, subject=False):
    content = u'' + content
    if subject:
        content = re.sub(r'\[.*\]\s?', '', content) # Remove name of mailing list from subject. eg: [DBpedia-Discussion] gets removed
        content = re.sub(r'(Re\s?:\s?|Fwd\s?:\s?)', '', content, flags=re.I) # Remove Reply or FWD eg: Re: [DBpedia-Discussion] Some Subject
    
    # Link Removals there are lot of cases so better to split them up for better understanding later on
    content = re.sub(r'https?:\/\/.*[\r\n\"<>()\s\/$]', ' ', content, re.MULTILINE)
    
    # Text Substitutions
    content = content.replace('&nbsp;', ' ').replace('&quot;', "'").replace('&amp;', '&').replace('&gt;', '>').replace('&lt;', '<').replace('&eq;', '=')# Removing Whitespace characters need to be extended to others as required
    
    # Content Stripping    
    content = re.sub(r'((\=\*){2,}|(\*\s?){2,}|(\#\s?){2,}|(_\s?){2,}|(\.\s?){2,}|(\-\s?){2,}|(\=\s?){3,})', '', content, flags=re.MULTILINE) # Remove Sequence of =*, *, #, _, =, -               
    content = re.sub(r'^>.*(\n|$)', ' ', content, flags=re.MULTILINE) # Remove Reply Sections

    # Normalize Text
    content = content.strip().replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    content = re.sub(r'\s{2,}', ' ', content, flags=re.MULTILINE) # Remove extra whitespaces
    
    return (u'' + content.strip()).encode('utf-8')

def print_full(string):
    print([c for c in string])

In [12]:
csv_reader = csv.reader(open('data/dbpedia-discussion-archive-no-sig.csv', 'r'))
csv_writer = csv.writer(open('data/dbpedia-discussion-archive.csv', 'wb'))
file_writer = open('data/dbpedia-discussion-archive-subjects.txt', 'w')

subjects = set()

for line in csv_reader:    
    subject = clean_content(line[3], True)    
    processed_msg = clean_content(line[4])
        
    csv_writer.writerow([line[0], line[1], line[2], subject, processed_msg])
    subjects.add(subject)

for subject in subjects:
    file_writer.write("%s\n" % subject)

In [30]:
content = "DBpedia installation. Richard [1] http://sites.wiwiss.fu-berlin.de/suhl/bizer/ng4j/semwebclient/ [2] http://seaborne.blogspot.com/2007/07/basic-federated-sparql-query.html [3] http://darq.sourceforge.net/ [4] http://docs.openlinksw.com/virtuoso/rdfiridereferencing.html(look at the examples in th"
content = re.sub(r'https?:\/\/\S+', ' ', content, re.MULTILINE)
print content

DBpedia installation. Richard [1]   [2]   [3]   [4]   at the examples in th
